# Pylops-gpu - combine pylops cupy operators and torch using TorchOperator

In this notebook we combine pylops and pylops-gpu to compute torch chains using pylops operators. More importantly, we consider the case where a GPU is available and we want to run the entire chain of operations without leaving the GPU. For this we take advantage of our cupy backend and its DLPack bindings (https://docs.cupy.dev/en/stable/reference/interoperability.html)

In [1]:
!nvidia-smi

Sat Jan  2 14:35:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2199.998
BogoMIPS:            4399.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

In [3]:
# install conda (https://towardsdatascience.com/conda-google-colab-75f7c867a522)
%env PYTHONPATH=

env: PYTHONPATH=


In [4]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745277_3 ...
installing: xz-5.2.4-h14c3975_4 ...
installing: yaml-0.1.7-had09818_2 ...
installing: zlib-1.2.11-ha838bed_2 ...
installing: libedit-3.1.20170329-h6b74fdf_2 ...
installing: readline-7.0-ha6073c6_4 ...
installing: sqlite-3.23.1-he433501_0 ...
installing: asn1crypto-0.24.0-py36_0 ...
installing: certifi-2018.4.16-py36_0 ...
installing: chardet-3.0.4-py36h0f667ec_1 ...
installing: idna-2.6-py36h82fb2a8_1 ...
installing: pycosat-0.6.3-py36h0a5515d_0 ...
installing: pycparser-2.18-py36hf9f622e_1 ...
installing: pysocks-1.6.8-py36_0 ...
installing: ruamel_yaml-0.15.37-py36h14c3975_2 ...
installing: six-1.11

--2021-01-02 14:35:03--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh [following]
--2021-01-02 14:35:03--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

     0K .......... .......... .......... .......... ..........  0% 57.6M 1s
    50K .......... .......... .......... .......... ..........  0%

In [5]:
%%bash
conda install --channel defaults conda python=3.6 --yes
conda update --channel defaults --all --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs: 
    - conda
    - python=3.6


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    setuptools-51.0.0          |   py36h06a4308_2         936 KB
    cffi-1.14.4                |   py36h261ae71_0         224 KB
    requests-2.25.1            |     pyhd3eb1b0_0          51 KB
    certifi-2020.12.5          |   py36h06a4308_0         144 KB
    brotlipy-0.7.0             |py36h27cfd23_1003         349 KB
    pycparser-2.20             |             py_2          94 KB
    cryptography-3.3.1         |   py36h3c74f83_0         638 KB
    ca-certificates-2020.12.8  |       h06a4308_0         128 KB
    xz-5.2.5                   |       h7b6447c_0         438 KB
    sqlite-3.33.0              |       h62c20be_0         2.0 MB
    openssl-1.1.1i             |       h27cfd23_0   

setuptools-51.0.0    |  936 KB | ########## | 100% 
cffi-1.14.4          |  224 KB | ########## | 100% 
requests-2.25.1      |   51 KB | ########## | 100% 
certifi-2020.12.5    |  144 KB | ########## | 100% 
brotlipy-0.7.0       |  349 KB | ########## | 100% 
pycparser-2.20       |   94 KB | ########## | 100% 
cryptography-3.3.1   |  638 KB | ########## | 100% 
ca-certificates-2020 |  128 KB | ########## | 100% 
xz-5.2.5             |  438 KB | ########## | 100% 
sqlite-3.33.0        |  2.0 MB | ########## | 100% 
openssl-1.1.1i       |  3.8 MB | ########## | 100% 
chardet-4.0.0        |  213 KB | ########## | 100% 
zlib-1.2.11          |  120 KB | ########## | 100% 
pyopenssl-20.0.1     |   48 KB | ########## | 100% 
pysocks-1.7.1        |   30 KB | ########## | 100% 
python-3.6.12        | 34.0 MB | ########## | 100% 
ld_impl_linux-64-2.3 |  645 KB | ########## | 100% 
tqdm-4.55.0          |   73 KB | ########## | 100% 
pip-20.3.3           |  2.0 MB | ########## | 100% 
yaml-0.2.5  

In [6]:
import sys
_ = (sys.path.append("/usr/local/lib/python3.6/site-packages"))

In [7]:
!conda install -c rapidsai -c conda-forge cusignal cudatoolkit=10.1 --yes

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cudatoolkit=10.1
    - cusignal


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    boost-1.72.0               |   py36h79e6602_1         338 KB  conda-forge
    boost-cpp-1.72.0           |       h8e57a91_0        21.8 MB  conda-forge
    bzip2-1.0.8                |       h516909a_3         398 KB  conda-forge
    ca-certificates-2020.12.5  |       ha878542_0         137 KB  conda-forge
    certifi-2020.12.5          |   py36h5fab9bb_0         143 KB  conda-forge
    conda-4.9.2                |   py36h5fab9bb_0         3.0 MB  conda-forge
    cudatoolkit-10.1.243       |       h6bb024c_0       347.4 MB
    cudnn-7.6.5.32             |       hc0a50b0_1       250.7 MB  conda-forge
    c

In [8]:
#!pip install cupy-cuda101
#!pip3 install cupy-cuda101==8.0.0rc1 --upgrade
!pip3 install cupy-cuda101==8.1.0 --upgrade
#!pip install pylops[advanced]
#!pip3 install git+https://github.com/PyLops/pylops.git
#!pip3 install pylops
!pip3 install pyFFTW
!conda install -c conda-forge pylops  --yes
!pip3 install git+https://github.com/PyLops/pylops-gpu

     |████████████████████████████████| 115.5 MB 33 kB/s 
     |████████████████████████████████| 2.6 MB 7.6 MB/s 
Solving environment: \ | / - \ | / - \ | / - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - pylops


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    fftw-3.3.8                 |nompi_hfc0cae8_1114         5.8 MB  conda-forge
    pyfftw-0.12.0              |   py36h3ea33d5_2         2.1 MB  conda-forge
    pylops-1.12.0              |   py36h1d69622_0         206 KB  conda-forge
    pywavelets-1.1.1           |   py36h68bb277_3         4.3 MB  conda-forge
    scikit-fmm-2019.1.30       |   py36h7c3b610_3          52 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        12.5 MB

The following NEW packages will be 

In [9]:
!rm -rf /usr/local/lib/python3.6/dist-packages/cupy*

In [10]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import timeit
import os
import numpy as np
import cupy as cp
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch
import pylops_gpu

from scipy.sparse.linalg import inv, lsqr
from scipy.sparse import rand

from pylops.utils import dottest
from pylops import LinearOperator
from pylops.basicoperators import *
from pylops.optimization.solver import cgls
from pylops.optimization.leastsquares import *

from pylops_gpu import TorchOperator
from pylops_gpu.utils.backend import device
from pylops_gpu.utils import dottest as gdottest
from pylops_gpu import Restriction as gRestriction

from torch.autograd import gradcheck

plt.style.use('seaborn-dark')

In [11]:
cp.__version__

'8.1.0'

In [12]:
#dev = device()
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('PyLops-gpu working on %s...' % dev)

torch.cuda.empty_cache()

PyLops-gpu working on cuda:0...


In [13]:
import torch

from torch.utils.dlpack import from_dlpack, to_dlpack
from pylops.utils import deps

if deps.cupy_enabled:
    import cupy as cp
else:
    cp = None


class _TorchOperator(torch.autograd.Function):
    """Wrapper class for PyLops operators into Torch functions

    The flag pylops is used to discriminate pylops and pylops-gpu operators;
    the former one requires the input to be converted into numpy.ndarray and
    the output to be converted back to torch.Tensor

    """
    @staticmethod
    def forward(ctx, x, forw, adj, pylops, device):
        ctx.forw = forw
        ctx.adj = adj
        ctx.pylops = pylops
        ctx.device = device

        # prepare input
        if ctx.pylops:
            if ctx.device == 'cpu':
                # bring x to cpu and numpy
                x = x.cpu().detach().numpy()
            else:
                # pass x to cupy using DLPack
                x = cp.fromDlpack(to_dlpack(x))
        # apply forward operator
        y = ctx.forw(x)

        # prepare output
        if ctx.pylops:
            if ctx.device == 'cpu':
                # move y to torch and device
                y = torch.from_numpy(y).to(ctx.device)
            else:
                # move y to torch and device
                y = from_dlpack(y.toDlpack())
        return y

    @staticmethod
    def backward(ctx, y):
        # prepare input
        if ctx.pylops:
            if ctx.device == 'cpu':
                y = y.cpu().detach().numpy()
            else:
                # pass x to cupy using DLPack
                y = cp.fromDlpack(to_dlpack(y))
        
        # apply adjoint operator
        x = ctx.adj(y)
        
        # prepare output
        if ctx.pylops:
            if ctx.device == 'cpu':
                x = torch.from_numpy(x).to(ctx.device)
            else:
              x = from_dlpack(x.toDlpack())                
        return  x, None, None, None, None


class TorchOperator():
    """Wrap a PyLops operator into a Torch function.

    This class can be used to wrap a pylops (or pylops-gpu) operator into a
    torch function. Doing so, users can mix native torch functions (e.g.
    basic linear algebra operations, neural networks, etc.) and pylops
    operators.

    Since all operators in PyLops are linear operators, a Torch function is
    simply implemented by using the forward operator for its forward pass
    and the adjont operator for its backward (gradient) pass.

    Parameters
    ----------
    Op : :obj:`pylops_gpu.LinearOperator` or :obj:`pylops.LinearOperator`
        PyLops operator
    batch : :obj:`bool`, optional
        Input has single sample (``False``) or batch of samples (``True``).
        If ``batch==False`` the input must be a 1-d Torch tensor,
        if `batch==False`` the input must be a 2-d Torch tensor with
        batches along the first dimension
    pylops : :obj:`bool`, optional
        ``Op`` is a pylops operator (``True``) or a pylops-gpu
        operator (``False``)
    device : :obj:`str`, optional
        Device to be used for output vectors when ``Op`` is a pylops operator

    Returns
    -------
    y : :obj:`torch.Tensor`
        Output array resulting from the application of the operator to ``x``.

    """
    def __init__(self, Op, batch=False, pylops=False, device='cpu'):
        self.pylops = pylops
        self.device = device
        if not batch:
            self.matvec = Op.matvec
            self.rmatvec = Op.rmatvec
        else:
            self.matvec = lambda x: Op.matmat(x, kfirst=True)
            self.rmatvec = lambda x: Op.rmatmat(x, kfirst=True)
        self.Top = _TorchOperator.apply

    def apply(self, x):
        """Apply forward pass to input vector

        Parameters
        ----------
        x : :obj:`torch.Tensor`
            Input array

        Returns
        -------
        y : :obj:`torch.Tensor`
            Output array resulting from the application of the operator to ``x``.

        """
        return self.Top(x, self.matvec, self.rmatvec,
                        self.pylops, self.device)


In [19]:
#pylops-gpu operator
n=100
perc_subsampling=0.4
nsub=int(np.round(n*perc_subsampling))
iava = np.sort(np.random.permutation(np.arange(n))[:nsub])

Rop = gRestriction(n, iava, dtype=torch.float64, device=dev)
x = torch.arange(n, dtype=torch.float64, requires_grad=True, device=dev)
y = TorchOperator(Rop, pylops=False).apply(x)

# gradient
v = torch.randn(nsub, dtype=torch.float64, device=dev)
y.backward(v, retain_graph=True)
print(x.grad)

inputs = (torch.randn(n, dtype=torch.double,requires_grad=True, device=dev))
test = gradcheck(TorchOperator(Rop).apply, inputs, eps=1e-6, atol=1e-4)
print(test)

tensor([ 0.0000,  0.0000, -0.1876,  0.8046,  0.0000,  0.0000,  0.0000,  0.0000,
         0.8418,  0.0000,  0.0000,  0.0000, -0.0947,  0.0000,  0.0000,  2.3580,
         0.0000, -1.4803,  0.0000,  0.0000,  0.0000,  0.3242,  0.0000, -0.3666,
         0.8174,  0.0000,  0.0000,  0.8215,  0.0000,  0.0000, -1.1196, -1.9174,
         0.0000,  0.0000,  0.0000,  0.0000,  1.0291, -1.9889, -1.2094,  0.0000,
         0.0000,  0.0000, -0.7478,  0.0000,  0.0000,  0.0000,  0.4308,  0.0000,
         0.1537,  0.0000,  0.0000,  0.0000,  0.0000, -1.0089,  0.7867,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.8167,  0.0000,  0.0000,  0.3421,
        -1.4783,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.5323,
         0.0000, -0.0163,  0.0000, -1.5529,  0.0000,  0.0000, -0.3472,  1.8992,
         0.0000,  0.0000, -0.9031, -2.6084, -0.0305,  1.4546,  0.0000,  0.0000,
         0.1467,  0.2310,  0.0000,  1.0688, -2.2056,  0.0000,  0.3828,  1.4593,
         0.0000,  0.0000,  0.5197, -1.59

In [20]:
# pylops+numpy operator
Rop = Restriction(n, iava, dtype=np.float64)
x = torch.arange(n, dtype=torch.float64, requires_grad=True)
y = TorchOperator(Rop, pylops=True).apply(x)

# gradient
v1 = v.clone().cpu()
y.backward(v1, retain_graph=True)
print(x.grad)

inputs = (torch.randn(n, dtype=torch.double,requires_grad=True))
Top = TorchOperator(Rop, pylops=True).apply
test = gradcheck(Top, inputs, eps=1e-6, atol=1e-4)
print(test)

tensor([ 0.0000,  0.0000, -0.1876,  0.8046,  0.0000,  0.0000,  0.0000,  0.0000,
         0.8418,  0.0000,  0.0000,  0.0000, -0.0947,  0.0000,  0.0000,  2.3580,
         0.0000, -1.4803,  0.0000,  0.0000,  0.0000,  0.3242,  0.0000, -0.3666,
         0.8174,  0.0000,  0.0000,  0.8215,  0.0000,  0.0000, -1.1196, -1.9174,
         0.0000,  0.0000,  0.0000,  0.0000,  1.0291, -1.9889, -1.2094,  0.0000,
         0.0000,  0.0000, -0.7478,  0.0000,  0.0000,  0.0000,  0.4308,  0.0000,
         0.1537,  0.0000,  0.0000,  0.0000,  0.0000, -1.0089,  0.7867,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.8167,  0.0000,  0.0000,  0.3421,
        -1.4783,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.5323,
         0.0000, -0.0163,  0.0000, -1.5529,  0.0000,  0.0000, -0.3472,  1.8992,
         0.0000,  0.0000, -0.9031, -2.6084, -0.0305,  1.4546,  0.0000,  0.0000,
         0.1467,  0.2310,  0.0000,  1.0688, -2.2056,  0.0000,  0.3828,  1.4593,
         0.0000,  0.0000,  0.5197, -1.59

In [21]:
# pylops+cupy operator
Rop = Restriction(n, iava, dtype=np.float64)
x = torch.arange(n, dtype=torch.float64, requires_grad=True, device=dev)
y = TorchOperator(Rop, pylops=True, device=dev).apply(x)

# gradient
y.backward(v, retain_graph=True)
print(x.grad)

inputs = (torch.randn(n, dtype=torch.double,requires_grad=True, device=dev))
test = gradcheck(TorchOperator(Rop, pylops=True, device=dev).apply, inputs, eps=1e-6, atol=1e-4)
print(test)

tensor([ 0.0000,  0.0000, -0.1876,  0.8046,  0.0000,  0.0000,  0.0000,  0.0000,
         0.8418,  0.0000,  0.0000,  0.0000, -0.0947,  0.0000,  0.0000,  2.3580,
         0.0000, -1.4803,  0.0000,  0.0000,  0.0000,  0.3242,  0.0000, -0.3666,
         0.8174,  0.0000,  0.0000,  0.8215,  0.0000,  0.0000, -1.1196, -1.9174,
         0.0000,  0.0000,  0.0000,  0.0000,  1.0291, -1.9889, -1.2094,  0.0000,
         0.0000,  0.0000, -0.7478,  0.0000,  0.0000,  0.0000,  0.4308,  0.0000,
         0.1537,  0.0000,  0.0000,  0.0000,  0.0000, -1.0089,  0.7867,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.8167,  0.0000,  0.0000,  0.3421,
        -1.4783,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.5323,
         0.0000, -0.0163,  0.0000, -1.5529,  0.0000,  0.0000, -0.3472,  1.8992,
         0.0000,  0.0000, -0.9031, -2.6084, -0.0305,  1.4546,  0.0000,  0.0000,
         0.1467,  0.2310,  0.0000,  1.0688, -2.2056,  0.0000,  0.3828,  1.4593,
         0.0000,  0.0000,  0.5197, -1.59